Project 2 Notebook Concert Data

In [1]:
#Dependencies
import pandas as pd
import numpy as np
import json
import requests
import time
from pandas import json_normalize
import os
import csv


In [2]:
#Reading in mtv genre information and converting it into dataframes
concert_total_path = "new_csv/concert_total.csv"

concert_data = pd.read_csv(concert_total_path)

In [3]:
concert_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 534 entries, 0 to 533
Data columns (total 6 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   name     534 non-null    object 
 1   date     534 non-null    object 
 2   tickets  532 non-null    float64
 3   gross    534 non-null    int64  
 4   month    534 non-null    int64  
 5   year     534 non-null    int64  
dtypes: float64(1), int64(3), object(2)
memory usage: 25.2+ KB


In [4]:
concert_data.head()

,name,date,tickets,gross,month,year
0,Adele,7/5/2016,31251.0,3402979,7,2016
1,Adele,7/10/2016,62704.0,6821680,7,2016
2,Adele,7/16/2016,30685.0,3376247,7,2016
3,Adele,7/20/2016,29043.0,3280806,7,2016
4,Adele,7/30/2016,29382.0,3270009,7,2016


In [5]:
#Grouping by month
concert_count_month = concert_data.groupby(["month"]).count()
concert_count_month

concert_count_month.to_csv("new_csv/concert_count_month.csv")

In [6]:
#Grouping by year
concert_count_year = concert_data.groupby(["year"]).count()
concert_count_year

concert_count_year.to_csv("new_csv/concert_count_year.csv")

In [7]:
#Grouping by year and month
concert_count_both = concert_data.groupby(["year", "month"]).count()
concert_count_both

concert_count_both.to_csv("new_csv/concert_count_both.csv")

Top Tour Section

In [8]:
import gmaps
# Import API key
from api_keys import g_key

In [9]:
#Reading in mtv genre information and converting it into dataframes
tour_path = "data/top_1-tour_with_locations.csv"

tour_data = pd.read_csv(tour_path)

In [10]:
tour_data.head()

,year,name,Date,City,State,Country,Venue,Revenue
0,2020,Celine Dion,1/8/2020,Jacksonville,Florida,USA,VyStar Veterans Memorial Arena,1912510
1,2020,Celine Dion,1/11/2020,Atlanta,GA,USA,State Farm Arena,2323672
2,2020,Celine Dion,1/13/2020,Nashville,TN,USA,Bridgestone Arena,2103662
3,2020,Celine Dion,1/15/2020,Tampa,Florida,USA,Amalie Arena,2254145
4,2020,Celine Dion,1/17/2020,Miami,Florida,USA,American Airlines Arena,5222838


In [11]:
#Inserting empty lat lng columns
tour_data["lat"] = ""
tour_data["lng"] = ""
tour_data.head()

,year,name,Date,City,State,Country,Venue,Revenue,lat,lng
0,2020,Celine Dion,1/8/2020,Jacksonville,Florida,USA,VyStar Veterans Memorial Arena,1912510,,
1,2020,Celine Dion,1/11/2020,Atlanta,GA,USA,State Farm Arena,2323672,,
2,2020,Celine Dion,1/13/2020,Nashville,TN,USA,Bridgestone Arena,2103662,,
3,2020,Celine Dion,1/15/2020,Tampa,Florida,USA,Amalie Arena,2254145,,
4,2020,Celine Dion,1/17/2020,Miami,Florida,USA,American Airlines Arena,5222838,,


In [12]:
params ={"key": g_key}

#Loop through cities and obtain coordinates for lat and long
for index, row in tour_data.iterrows():
    base_url = "https://maps.googleapis.com/maps/api/geocode/json"
    city = row["City"]
    state = row["State"]
    country = row["Country"]
    
    #Add these items to search query
    params["address"] = city
    params["components = state"] = state
    params["components = country"] = country
    
    #Make request
    cities_lat_lng = requests.get(base_url, params=params)
    cities_lat_lng = cities_lat_lng.json()
    
    #Overwrite lat lng values with center of the city coordinates
    try:
        tour_data.loc[index, "lat"] = cities_lat_lng["results"][0]["geometry"]["location"]["lat"]
        tour_data.loc[index, "lng"] = cities_lat_lng["results"][0]["geometry"]["location"]["lng"]
    except:
        print(f"Data not available for {city}, {country}")

In [13]:
tour_data.head()

,year,name,Date,City,State,Country,Venue,Revenue,lat,lng
0,2020,Celine Dion,1/8/2020,Jacksonville,Florida,USA,VyStar Veterans Memorial Arena,1912510,30.3322,-81.6557
1,2020,Celine Dion,1/11/2020,Atlanta,GA,USA,State Farm Arena,2323672,33.749,-84.388
2,2020,Celine Dion,1/13/2020,Nashville,TN,USA,Bridgestone Arena,2103662,36.1627,-86.7816
3,2020,Celine Dion,1/15/2020,Tampa,Florida,USA,Amalie Arena,2254145,27.9506,-82.4572
4,2020,Celine Dion,1/17/2020,Miami,Florida,USA,American Airlines Arena,5222838,25.7617,-80.1918


In [18]:
tour_data.to_json('tour_data_json', orient = "index")

In [19]:
tour_data_json